In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import gpflow
import gpflux
import sklearn
from sklearn import model_selection
from sklearn import preprocessing
from sklearn import metrics
from sklearn import svm

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv("flight price predicting.csv")
tf.keras.backend.set_floatx("float64")

In [4]:
stops = df['stops']
stops[stops=='zero']=0
stops[stops=='one']=1
stops[stops=='two_or_more']=2

In [5]:
df = pd.get_dummies(df.drop(['stops'], axis=1))

In [6]:
df = df.drop(['airline_AirAsia','departure_time_Afternoon','class_Business'], axis=1)
df['stops'] = stops

In [7]:
X = np.array(df.drop(['Unnamed: 0','price'], axis=1))
Y = np.array(df['price'])

In [8]:
X = np.reshape(X,(-1,14))
Y = np.reshape(Y, (-1,1))

In [9]:
X_train = X[0:7000]
X_val = X[7000:8500]
X_test = X[8500:10000]

In [10]:
Y_train = Y[0:7000]
Y_val = Y[7000:8500]
Y_test = Y[8500:10000]

In [11]:
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [12]:
n = len(Y_train)

In [132]:
kernels = [gpflow.kernels.Cosine(), gpflow.kernels.Matern12(), gpflow.kernels.Matern32(), gpflow.kernels.Matern52(),
           gpflow.kernels.RBF(), gpflow.kernels.RationalQuadratic(), 
           gpflow.kernels.Periodic(base_kernel=gpflow.kernels.RBF())]

In [133]:
best_mse1 = 100000000
best_model1 = None
mae1 = None
best_kernel = None

In [134]:
for k in kernels:
    for i in range(0,3):
        
        kernel1 = k
        inducing_points1 = gpflow.inducing_variables.InducingPoints(np.random.random((256, 14)))
        layer1 = gpflux.layers.GPLayer(kernel1,inducing_points1,num_data=n,num_latent_gps=1,
                                  mean_function=gpflow.mean_functions.Zero())
        likelihood = gpflux.layers.LikelihoodLayer(gpflow.likelihoods.Gaussian())
        gp = gpflux.models.DeepGP([layer1],likelihood)
        model = gp.as_training_model()
        model.compile(tf.optimizers.Adam(0.01))
        model.fit({"inputs":X_train,"targets":Y_train},epochs=150,verbose=0)
        model = gp.as_prediction_model()
        out_val = model(X_val)
        pred_val = out_val.f_mean.numpy().squeeze()
        mse_val = metrics.mean_squared_error(Y_val,pred_val)
        mae_val = metrics.mean_absolute_error(Y_val,pred_val)
        if(mse_val < best_mse1):
            best_mse1 = mse_val
            best_model1 =model
            mae1 = mae_val
            best_kernel = k

In [135]:
best_kernel

name,class,transform,prior,trainable,shape,dtype,value
Periodic.base_kernel.variance,Parameter,Softplus,,True,(),float64,814.403
Periodic.base_kernel.lengthscales,Parameter,Softplus,,True,(),float64,1.3211
Periodic.period,Parameter,Softplus,,True,(),float64,1.39259


In [136]:
best_mse1

52490367.04621509

In [137]:
mae1

4973.18546246302

In [154]:
out_test = best_model1(X_test)
pred_test = out_test.f_mean.numpy().squeeze()
mse_test1 = metrics.mean_squared_error(Y_test,pred_test)
mae_test1 = metrics.mean_absolute_error(Y_test,pred_test)

In [155]:
mse_test1

56354818.09977449

In [156]:
mae_test1

5194.495738223249

In [13]:
best_mse2 = 1000000000
best_model2 = None
mae2 = None

In [14]:
for i in range(0,2):
    kernel1 = gpflow.kernels.Matern32()+gpflow.kernels.Matern52()
    inducing_points1 = gpflow.inducing_variables.InducingPoints(np.random.random((256, 14)))
    layer1 = gpflux.layers.GPLayer(kernel1,inducing_points1,num_data=n,num_latent_gps=14)
    kernel2 = gpflow.kernels.Matern32()+gpflow.kernels.Matern52()
    inducing_points2 = gpflow.inducing_variables.InducingPoints(np.random.random((128, 14)))
    layer2 = gpflux.layers.GPLayer(kernel2,inducing_points2,num_data =n,num_latent_gps=1,mean_function=gpflow.mean_functions.Zero())
    likelihood = gpflux.layers.LikelihoodLayer(gpflow.likelihoods.Gaussian())
    deep_gp = gpflux.models.DeepGP([layer1, layer2],likelihood)
    model = deep_gp.as_training_model()
    model.compile(tf.optimizers.Adam(0.01))
    model.fit({"inputs":X_train,"targets":Y_train},epochs=150,verbose=0)
    model = deep_gp.as_prediction_model()
    out_val = model(X_val)
    pred_val = out_val.f_mean.numpy().squeeze()
    mse_val = metrics.mean_squared_error(Y_val,pred_val)
    mae_val = metrics.mean_absolute_error(Y_val,pred_val)
    if(mse_val < best_mse2):
        best_mse2 = mse_val
        best_model2 = model
        mae2 = mae_val   

In [15]:
best_mse2

31177263.200669922

In [16]:
mae2

3356.8462461151917

In [17]:
out_test = best_model2(X_test)
pred_test = out_test.f_mean.numpy().squeeze()
mse_test2 = metrics.mean_squared_error(Y_test,pred_test)
mae_test2 = metrics.mean_absolute_error(Y_test,pred_test)

In [18]:
mse_test2

35182097.30740546

In [19]:
mae_test2

3582.694364021823

In [20]:
best_mse3 = 1000000000
best_model3 = None
mae3 = None

In [21]:
for i in range(0,2):
    kernel1 = gpflow.kernels.RBF()*gpflow.kernels.Matern52()
    inducing_points1 = gpflow.inducing_variables.InducingPoints(np.random.random((256, 14)))
    layer1 = gpflux.layers.GPLayer(kernel1,inducing_points1,num_data=n,num_latent_gps=14)
    kernel2 = gpflow.kernels.RBF()*gpflow.kernels.Matern52()
    inducing_points2 = gpflow.inducing_variables.InducingPoints(np.random.random((128, 14)))
    layer2 = gpflux.layers.GPLayer(kernel2,inducing_points2,num_data =n,num_latent_gps=1,mean_function=gpflow.mean_functions.Zero())
    likelihood = gpflux.layers.LikelihoodLayer(gpflow.likelihoods.Gaussian())
    deep_gp = gpflux.models.DeepGP([layer1, layer2],likelihood)
    model = deep_gp.as_training_model()
    model.compile(tf.optimizers.Adam(0.01))
    model.fit({"inputs":X_train,"targets":Y_train},epochs=150,verbose=0)
    model = deep_gp.as_prediction_model()
    out_val = model(X_val)
    pred_val = out_val.f_mean.numpy().squeeze()
    mse_val = metrics.mean_squared_error(Y_val,pred_val)
    mae_val = metrics.mean_absolute_error(Y_val,pred_val)
    if(mse_val < best_mse3):
        best_mse3 = mse_val
        best_model3 = model
        mae3 = mae_val

In [22]:
best_mse3

29423876.358454034

In [23]:
mae3

3310.7740737339136

In [24]:
out_test = best_model3(X_test)
pred_test = out_test.f_mean.numpy().squeeze()
mse_test3 = metrics.mean_squared_error(Y_test,pred_test)
mae_test3 = metrics.mean_absolute_error(Y_test,pred_test)

In [25]:
mse_test3

34691688.52204659

In [26]:
mae_test3

3603.9753112740186

In [106]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError
from tensorflow.keras.optimizers import Adam

In [174]:
best_model4 = None
best_mse4 = 100000000
mae4 = None 

In [175]:
for i in range(0,3):
    model = Sequential([InputLayer(14,),Dense(32, activation='relu'),Dense(16, activation='relu'), Dense(1)])
    model.compile(optimizer=Adam(learning_rate=0.001), loss=MeanSquaredError(),
              metrics = [MeanAbsoluteError()])
    model.fit(X_train,Y_train,epochs=150, batch_size=128, validation_data=(X_val,Y_val), verbose=0)
    mse_val, mae_val = model.evaluate(X_val,Y_val)
    if(mse_val < best_mse4):
        best_mse4 = mse_val
        mae4 = mae_val
        best_model4 = model

47/47 [==============================] - 0s 2ms/step - loss: 42364678.7137 - mean_absolute_error: 4431.1146


In [176]:
best_mse4

42364678.7137116

In [177]:
mae4

4431.114609537754

In [178]:
mse_test4 = metrics.mean_squared_error(Y_test, best_model4.predict(X_test))
mae_test4 = metrics.mean_absolute_error(Y_test, best_model4.predict(X_test))

47/47 [==============================] - 0s 2ms/step


In [179]:
mse_test4

46099064.6234641

In [180]:
mae_test4

4667.179386764309

In [111]:
model5 = sklearn.linear_model.LinearRegression()

In [112]:
model5.fit(X_train, Y_train)

LinearRegression()

In [113]:
pred_val = model5.predict(X_val)

In [114]:
mse5 = metrics.mean_squared_error(Y_val,pred_val)

In [115]:
mae5 = metrics.mean_absolute_error(Y_val,pred_val)

In [116]:
mse5

49683951.80362598

In [117]:
mae5

4557.8224521302855

In [160]:
pred_test = model5.predict(X_test)
mse_test5 = metrics.mean_squared_error(Y_test,pred_test)
mae_test5 = metrics.mean_absolute_error(Y_test,pred_test)

In [161]:
mse_test5

50396230.30277953

In [162]:
mae_test5

4756.333378526396

In [145]:
Cs = [0.001, 0.01, 0.1, 1, 10, 100]
gammas = [0.001, 0.01, 0.1, 1, 10, 100]

In [146]:
best_mse6 = 1000000000
mae6 = None
best_C = None
best_gamma = None
best_model6 = None

In [147]:
for C in Cs:
    for gamma in gammas:
        model = svm.SVR(kernel='rbf', gamma=gamma, C = C)
        model.fit(X_train, Y_train)
        mse_val = metrics.mean_squared_error(Y_val, model.predict(X_val))
        mae_val = metrics.mean_absolute_error(Y_val, model.predict(X_val))
        if (mse_val < best_mse6):
            best_mse6 = mse_val
            best_C = C
            best_gamma = gamma
            best_model6 = model
            mae6 = mae_val

In [148]:
best_C

100

In [149]:
best_gamma

0.1

In [150]:
best_mse6

116932961.05863076

In [151]:
mae6

6305.838578537671

In [163]:
model = svm.SVR(kernel='rbf', gamma=best_gamma, C = best_C)
model.fit(X_train, Y_train)

SVR(C=100, gamma=0.1)

In [164]:
mse_test6 = metrics.mean_squared_error(Y_test, model.predict(X_test))
mae_test6 = metrics.mean_absolute_error(Y_test, model.predict(X_test))

In [165]:
mse_test6

127395317.19212858

In [166]:
mae_test6

6767.939052808309